In [ ]:
import polars as pl
from polars import col

def get_bundle_header_ldf():
    bundle_header_path = "../brontes-exports/mev/bundles/header/03-28/02-34-31.parquet"
    return pl.scan_parquet(bundle_header_path)

def get_cex_dex_ldf():
    cex_dex_path = "../brontes-exports/mev/bundles/header/03-28/01-49-41.parquet"
    cex_dex_ldf = pl.scan_parquet(cex_dex_path)
    #return get_bundle_header_ldf().join(cex_dex_ldf, on="tx_hash", how="inner")
    cex_dex_ldf

def get_sandwich_ldf():
    sandwich_path = "../brontes-exports/mev/bundles/sandwich/03-28/02-34-31.parquet"
    sandwich_ldf = pl.scan_parquet(sandwich_path)
    return get_bundle_header_ldf().join(
        sandwich_ldf,
        left_on="tx_hash",
        right_on=col("frontrun_tx_hashes").arr.first(),
        how="inner",
    )

def get_jit_ldf():
    jit_path = "../brontes-exports/mev/bundles/jit/03-28/02-34-31.parquet"
    jit_ldf = pl.scan_parquet(jit_path)
    return get_bundle_header_ldf().join(
        jit_ldf,
        left_on="tx_hash",
        right_on=col("frontrun_tx_hashes").arr.first(),
        how="inner",
    )

def get_jit_sandwich_ldf():
    jit_sandwich_path = "../brontes-exports/mev/bundles/jit-sandwich/03-28/02-34-31.parquet"
    jit_sandwich_ldf = pl.scan_parquet(jit_sandwich_path)
    return get_bundle_header_ldf().join(
        jit_sandwich_ldf,
        left_on="tx_hash",
        right_on=col("frontrun_tx_hashes").arr.first(),
        how="inner",
    )

def get_liquidation_ldf():
    liquidation_path = "../brontes-exports/mev/bundles/liquidation/03-28/02-34-31.parquet"
    liquidation_ldf = pl.scan_parquet(liquidation_path)
    return get_bundle_header_ldf().join(
        liquidation_ldf,
        left_on="tx_hash",
        right_on="liquidation_tx_hash",
        how="inner",
    )

def get_atomic_arb_ldf():
    atomic_arb_path = "../brontes-exports/mev/bundles/atomic-arb/03-28/02-34-31.parquet"
    atomic_arb_ldf = pl.scan_parquet(atomic_arb_path)
    return get_bundle_header_ldf().join(atomic_arb_ldf, on="tx_hash", how="inner")

def get_searcher_tx_ldf():
    searcher_tx_path = "../brontes-exports/mev/bundles/searcher-tx/03-28/02-34-31.parquet"
    searcher_tx_ldf = pl.scan_parquet(searcher_tx_path)
    return get_bundle_header_ldf().join(searcher_tx_ldf, on="tx_hash", how="left")

def group_by_mev_contract_and_type(ldf):
    return (
        ldf.group_by(["mev_type"])
        .agg(
            [
                pl.col("tx_index").median().alias("median_tx_index"),
                #pl.col("eoa").unique().alias("unique_eoas"),
                pl.col("profit_usd").sum().alias("total_profit"),
                pl.col("profit_usd").mean().alias("profit_mean"),
                pl.col("bribe_usd").sum().alias("total_bribed"),
                pl.col("bribe_usd").mean().alias("bribe_mean"),
                (pl.col("bribe_usd").sum() + pl.col("profit_usd").sum()).alias("total_revenue"),
                pl.col("mev_contract").count().alias("bundle_count"),
            ]
        )
        .sort("total_profit", descending=True, nulls_last=True)
    )


def group_by_fund(ldf):
    return (
        ldf.lazy().group_by("fund")
        .agg(
            [
                pl.col("profit_usd").sum().alias("total_profit"),
                pl.col("profit_usd").mean().alias("profit_mean"),
                pl.col("bribe_usd").sum().alias("total_bribed"),
                pl.col("bribe_usd").mean().alias("bribe_mean"),
                (pl.col("bribe_usd").sum() + pl.col("profit_usd").sum()).alias("total_revenue"),
                pl.col("mev_contract").count().alias("bundle_count"),
            ]
        )
        .sort("total_profit", descending=True, nulls_last=True)
    )
